In [2]:
import os
import json
import torch
from PIL import Image
from pathlib import Path
import pandas as pd

In [3]:
home_dir = "/root/CVPDL/hw3"
os.chdir(home_dir)

### Data Preprocess

In [6]:
# select the first 20 images contains less box
def select_images(data, n_head) -> pd.DataFrame:    
    df = pd.DataFrame(data)
    grouped_df = df.groupby('main_creature', group_keys=False)
    sorted_df = grouped_df.apply(lambda x: x.sort_values(by="n_box", ascending=True).head(n_head))
    
    return sorted_df

def normalize_box(df):
    for i, boxes in df.iterrows():
        for j, box in enumerate(boxes["bbox"]):
            df.loc[i, "bbox"][j][0] = box[0] / df.at[i, "width"]
            df.loc[i, "bbox"][j][1] = box[1] / df.at[i, "height"]
            df.loc[i, "bbox"][j][2] = box[2] / df.at[i, "width"] + df.at[i, "bbox"][j][0]
            df.loc[i, "bbox"][j][3] = box[3] / df.at[i, "height"] + df.at[i, "bbox"][j][1]
    return df

def data_preprocess(data):
    df = select_images(data, n_head=20)
    df = normalize_box(df)
    return df.to_dict(orient='records')

In [7]:
in_file = Path("lib/config/generated_caption_blip2_t5.json")
with in_file.open("r") as f:
    data = json.load(f)

data = data_preprocess(data)


In [8]:
# save original data
data_dir = Path("dataset/cvpdl/train")
o_dir = Path("dataset/inpainted_images")
if not o_dir.exists():
    o_dir.mkdir()

for idx, instance in enumerate(data):
    image = Image.open(data_dir / instance["file_name"])
    image.save(o_dir / "{}.jpg".format(idx + 1))

### Modeling - Image Grounding 

In [70]:
def get_model_inpuat(data, data_dir, prompt_template="prompt_wtemp1"):
    prompt = data[prompt_template]
    phrases = data["creature"]
    image = Image.open(data_dir / data["file_name"])
    boxes = data["bbox"]
    return prompt, phrases, image, boxes

def inference_one_instance_image_grounding(pipe, prompt, phrases, input_image, boxes, o_dir, generate_idx):
    images = pipe(
        prompt=prompt,
        gligen_phrases=phrases,
        gligen_inpaint_image=input_image,
        gligen_boxes=boxes,
        gligen_scheduled_sampling_beta=1,
        output_type="pil",
        num_inference_steps=50,
    ).images

    images[0].save(o_dir / "{}.jpg".format(generate_idx))

In [ ]:
import torch
from diffusers import StableDiffusionGLIGENPipeline
from diffusers.utils import load_image

# Insert objects described by text at the region defined by bounding boxes
pipe = StableDiffusionGLIGENPipeline.from_pretrained(
    "masterful/gligen-1-4-inpainting-text-box", variant="fp16", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

In [72]:
data_dir = Path("dataset") / "cvpdl" / "train"
template_id = 1
o_dir = Path("dataset/cvpdl_gligen_mix/train") / ("image_grounding" + template_id)
prompt_template = "prompt_wtemp" + template_id
for idx, instance in enumerate(data):
    prompt, phrases, input_image, boxes = get_model_inpuat(instance, data_dir, prompt_template)
    inference_one_instance_image_grounding(pipe, prompt, phrases, input_image, boxes, o_dir, generate_idx=idx+1)

{'file_name': 'IMG_8461_jpg.rf.c1f3ef25fac44501963625279e8ca614.jpg', 'height': 768, 'width': 1024, 'creature': ['fish'], 'bbox': [[0.2490234375, 0.2513020833333333, 0.35546875, 0.4700520833333333]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'a fish is swimming in the water', 'prompt_wtemp1': 'a fish is swimming in the water, creature: fish', 'prompt_wtemp2': 'a fish is swimming in the water, creature: fish, height: 768, width: 1024'}
prompt_wtemp1


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2375_jpeg_jpg.rf.ed73b2179f3b96fe3088456fc43a2cf5.jpg', 'height': 1024, 'width': 768, 'creature': ['fish'], 'bbox': [[0.07291666666666667, 0.353515625, 0.9908854166666666, 0.703125]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'a fish swimming in an aquarium with rocks', 'prompt_wtemp1': 'a fish swimming in an aquarium with rocks, creature: fish', 'prompt_wtemp2': 'a fish swimming in an aquarium with rocks, creature: fish, height: 1024, width: 768'}
prompt_wtemp1


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_8549_jpg.rf.ee5f8f7fa2cae6f1e528e1b350ad4046.jpg', 'height': 768, 'width': 1024, 'creature': ['fish'], 'bbox': [[0.1962890625, 0.4674479166666667, 0.498046875, 0.7109375]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'clownfish in the aquarium at the san diego aquarium', 'prompt_wtemp1': 'clownfish in the aquarium at the san diego aquarium, creature: fish', 'prompt_wtemp2': 'clownfish in the aquarium at the san diego aquarium, creature: fish, height: 768, width: 1024'}
prompt_wtemp1


...

  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2655_jpeg_jpg.rf.500bca587eb20e6e80acbea6f60ebd40.jpg', 'height': 1024, 'width': 768, 'creature': ['stingray', 'stingray'], 'bbox': [[0.3671875, 0.224609375, 0.5572916666666666, 0.3369140625], [0.3580729166666667, 0.41015625, 0.5598958333333334, 0.5263671875]], 'n_box': 2, 'main_creature': 'stingray', 'gerated_caption': 'a ray fish swimming in the water', 'prompt_wtemp1': 'a ray fish swimming in the water, creature: stingray', 'prompt_wtemp2': 'a ray fish swimming in the water, creature: stingray, height: 1024, width: 768'}
prompt_wtemp1


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2619_jpeg_jpg.rf.62b48e73de77f2d89e512df79ebb3041.jpg', 'height': 1024, 'width': 768, 'creature': ['stingray', 'fish'], 'bbox': [[0.28125, 0.380859375, 0.7252604166666667, 0.49609375], [0.38671875, 0.658203125, 0.5052083333333334, 0.705078125]], 'n_box': 2, 'main_creature': 'stingray', 'gerated_caption': 'a stingray swimming in an aquarium', 'prompt_wtemp1': 'a stingray swimming in an aquarium, creature: stingray', 'prompt_wtemp2': 'a stingray swimming in an aquarium, creature: stingray, height: 1024, width: 768'}
prompt_wtemp1


  0%|          | 0/50 [00:00<?, ?it/s]

### Modeling - Text Grounding

In [73]:
def inference_one_instance_text_grounding(pipe, prompt, phrases,  boxes, o_dir, generate_idx):
    images = pipe(
        prompt=prompt,
        gligen_phrases=phrases,
        gligen_boxes=boxes,
        gligen_scheduled_sampling_beta=1,
        output_type="pil",
        num_inference_steps=50,
    ).images

    images[0].save(o_dir / "{}.jpg".format(generate_idx))                               

In [74]:
# Generate an image described by the prompt and
# insert objects described by text at the region defined by bounding boxes
pipe = StableDiffusionGLIGENPipeline.from_pretrained(
    "masterful/gligen-1-4-generation-text-box", variant="fp16", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

safety_checker/model.fp16.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

pytorch_model.fp16.bin:   0%|          | 0.00/608M [00:00<?, ?B/s]

pytorch_model.fp16.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/usr/local/lib/anaconda3/envs/hw3/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [79]:
data_dir = Path("dataset") / "cvpdl" / "train"
template_id = str(2)
o_dir = Path("dataset") / ("text_grounding" + template_id)
if not o_dir.exists():
    o_dir.mkdir()
prompt_template = "prompt_wtemp" + template_id
for idx, instance in enumerate(data):
    prompt, phrases, input_image, boxes = get_model_inpuat(instance, data_dir, prompt_template)
    inference_one_instance_text_grounding(pipe, prompt, phrases, boxes, o_dir, generate_idx=idx+1)

{'file_name': 'IMG_8461_jpg.rf.c1f3ef25fac44501963625279e8ca614.jpg', 'height': 768, 'width': 1024, 'creature': ['fish'], 'bbox': [[0.2490234375, 0.2513020833333333, 0.35546875, 0.4700520833333333]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'a fish is swimming in the water', 'prompt_wtemp1': 'a fish is swimming in the water, creature: fish', 'prompt_wtemp2': 'a fish is swimming in the water, creature: fish, height: 768, width: 1024'}
prompt_wtemp2


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2375_jpeg_jpg.rf.ed73b2179f3b96fe3088456fc43a2cf5.jpg', 'height': 1024, 'width': 768, 'creature': ['fish'], 'bbox': [[0.07291666666666667, 0.353515625, 0.9908854166666666, 0.703125]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'a fish swimming in an aquarium with rocks', 'prompt_wtemp1': 'a fish swimming in an aquarium with rocks, creature: fish', 'prompt_wtemp2': 'a fish swimming in an aquarium with rocks, creature: fish, height: 1024, width: 768'}
prompt_wtemp2


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_8549_jpg.rf.ee5f8f7fa2cae6f1e528e1b350ad4046.jpg', 'height': 768, 'width': 1024, 'creature': ['fish'], 'bbox': [[0.1962890625, 0.4674479166666667, 0.498046875, 0.7109375]], 'n_box': 1, 'main_creature': 'fish', 'gerated_caption': 'clownfish in the aquarium at the san diego aquarium', 'prompt_wtemp1': 'clownfish in the aquarium at the san diego aquarium, creature: fish', 'prompt_wtemp2': 'clownfish in the aquarium at the san diego aquarium, creature: fish, height: 768, width: 1024'}
prompt_wtemp2


...

  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2655_jpeg_jpg.rf.500bca587eb20e6e80acbea6f60ebd40.jpg', 'height': 1024, 'width': 768, 'creature': ['stingray', 'stingray'], 'bbox': [[0.3671875, 0.224609375, 0.5572916666666666, 0.3369140625], [0.3580729166666667, 0.41015625, 0.5598958333333334, 0.5263671875]], 'n_box': 2, 'main_creature': 'stingray', 'gerated_caption': 'a ray fish swimming in the water', 'prompt_wtemp1': 'a ray fish swimming in the water, creature: stingray', 'prompt_wtemp2': 'a ray fish swimming in the water, creature: stingray, height: 1024, width: 768'}
prompt_wtemp2


  0%|          | 0/50 [00:00<?, ?it/s]

{'file_name': 'IMG_2619_jpeg_jpg.rf.62b48e73de77f2d89e512df79ebb3041.jpg', 'height': 1024, 'width': 768, 'creature': ['stingray', 'fish'], 'bbox': [[0.28125, 0.380859375, 0.7252604166666667, 0.49609375], [0.38671875, 0.658203125, 0.5052083333333334, 0.705078125]], 'n_box': 2, 'main_creature': 'stingray', 'gerated_caption': 'a stingray swimming in an aquarium', 'prompt_wtemp1': 'a stingray swimming in an aquarium, creature: stingray', 'prompt_wtemp2': 'a stingray swimming in an aquarium, creature: stingray, height: 1024, width: 768'}
prompt_wtemp2


  0%|          | 0/50 [00:00<?, ?it/s]

### Save metadata

In [9]:
def save_gererated_image_metadata(data, grounding_type, template_id):
    i_path = Path("dataset/cvpdl/annotations/train.json")
    with i_path.open("r") as f:
        data_format = json.load(f)

    category_to_id = {category["name"]:category["id"] for category in data_format["categories"]}
    image_related, annotation_related = [], []
    image_start_id = 448
    annotation_start_id = 3323
    anno_idx = 0
    for idx, instance in enumerate(data):
        img_metadata = {}
        img_metadata["file_name"] = "{}{}/{}.jpg".format(grounding_type, template_id, idx + 1)
        img_metadata["height"] = instance["height"]
        img_metadata["width"] = instance["width"]
        img_metadata["id"] = 448 + idx
        img_metadata["license"] = 1
        img_metadata["date_captured"] = "None"
        image_related.append(img_metadata)
        
        for category, box in zip(instance["creature"], instance["bbox"]):
            annotation_metadata = {}
            annotation_metadata["id"] = annotation_start_id + anno_idx
            annotation_metadata["image_id"] = image_start_id + idx
            annotation_metadata["category_id"] = category_to_id[category]
            # gligen to coco format (512, 512) -> original size
            x1 = box[0] * 512
            y1 = box[1] * 512
            x2 = box[2] * 512 - x1
            y2 = box[3] * 512 - y1
            resized_box = [int(x1), int(y1), int(x2), int(y2)]
            annotation_metadata["bbox"] = resized_box
            annotation_metadata["area"] = int(x2) * int(y2)
            annotation_metadata["segmentation"] = []
            annotation_metadata["iscrowd"] = 0
            annotation_related.append(annotation_metadata)
            anno_idx += 1
    
    data_format["images"] += image_related
    data_format["annotations"] += annotation_related
    print("Numbers of images : {}".format(len(data_format["images"])))
    print("Numbers of boxes : {}".format(len(data_format["annotations"])))
    print(data_format["annotations"][3450]["category_id"])
    o_path = Path("dataset/cvpdl_gligen_mix/annotations/train.json")
    with o_path.open("w") as f:
        json.dump(data_format, f, indent=4)
    

In [10]:
grounding_type = "image_grounding"
template_id = 1
save_gererated_image_metadata(data, grounding_type, template_id)

Numbers of images : 588
Numbers of boxes : 3618
2
